## 1.3 캔들 사용
인공지능으로 숫자를 예측하는 실습을 하면서 파이썬 텍스트 모드와 파이썬 노트북 모드에서 케라스 사용법을 익힙시다.

### 1.3.0 캔들 테스트

In [3]:
:dep candle-core
:dep candle-nn


### 1.3.2 텍스트 모드 실습

1. 실습 진행에서 가장 먼저 할 일은 캔들 패키지 및 다른 필요한 패키지의 관련 기능을 임포트하는 겁니다.

In [35]:
use candle_core::{DType, Device, Result, Tensor};
use candle_nn::{linear, AdamW, Linear, Module, Optimizer, ParamsAdamW, VarBuilder, VarMap};

2. 학습 및 평가에 사용할 전체 데이터를 지정합니다. 

In [98]:
fn gen_xy_data(x: Vec<f32>) -> Result<(Tensor, Tensor)> {
    println!("[gen_data]");
    let x = Tensor::new(x, &Device::Cpu); //vec1
    println!("Ok(x): {:?}", x); // show with Ok()
    let x = x?; // unwrap of Ok()
    println!("x: {:?}", x); // show the property of x
    println!("x.to_vec1::<f32>()?: {:?}", x.to_vec1::<f32>()?);
    let y = ((x.clone() * 2.0)? + 1.0)?; // Note: use of clone(), ? 
    println!("y.to_vec1::<f32>()?: {:?}", y.to_vec1::<f32>()?); // Note: use of <f32>
    println!();
    Ok((x, y))
}

In [109]:
fn print_model(model: &Linear) -> Result<()> {
    println!("model: {:?}", model);    
    println!("model.weight(): {:?}", model.weight());
    println!("model.weight().to_vec2::<f32>()?: {:?}", model.weight().to_vec2::<f32>()?);
    println!("model.bias(): {:?}", model.bias()); //unwrap  
    println!("model.bias().unwrap(): {:?}", model.bias().unwrap()); //unwrap 
    println!("model.bias().unwrap().to_vec1::<f32>()?: {:?}", model.bias().unwrap().to_vec1::<f32>()?); //unwrap  
    Ok(())
}

fn main() -> Result<()> {
    let (x, y) = gen_xy_data(vec![0_f32, 1.])?;
    
    println!("[Define and Test Model]");
    let varmap = VarMap::new();
    let vb = VarBuilder::from_varmap(&varmap, DType::F32, &Device::Cpu);
    let model = linear(1, 1, vb.pp("linear"))?;
    print_model(&model)?;

    println!("x.dims1(): {:?}", x.dims1()?);
    let x = x.reshape((x.dims1()?,1))?; // reshpe to provide for model
    let y = y.reshape((y.dims1()?,1))?; 
    let y_pred = model.forward(&x)?;
    println!("{:?}", y_pred);
    println!("y_pred.to_vec2::<f32>()?: {:?}", y_pred.to_vec2::<f32>()?);
    println!();

    println!("[Training]");
    let params = ParamsAdamW {
        lr: 0.1,
        ..Default::default()
    };
    let mut opt = AdamW::new(varmap.all_vars(), params)?;

    for step in 0..=1000 {
        let y_pred = model.forward(&x)?;
        let loss = y_pred.sub(&y)?.sqr()?.sum_all()?;
        opt.backward_step(&loss)?;
        if step % 100 == 0 {
            println!("{step} {}", loss.to_vec0::<f32>()?);
        }
    }
    print_model(&model);

    Ok(())
}

main()

[gen_data]
Ok(x): Ok(Tensor[0, 1; f32])
x: Tensor[0, 1; f32]
x.to_vec1::<f32>()?: [0.0, 1.0]
y.to_vec1::<f32>()?: [1.0, 3.0]

[Define and Test Model]
model: Linear { weight: Tensor[dims 1, 1; f32], bias: Some(Tensor[-0.021972418; f32]) }
model.weight(): Tensor[dims 1, 1; f32]
model.weight().to_vec2::<f32>()?: [[1.3844671]]
model.bias(): Some(Tensor[-0.021972418; f32])
model.bias().unwrap(): Tensor[-0.021972418; f32]
model.bias().unwrap().to_vec1::<f32>()?: [-0.021972418]
x.dims1(): 2
Tensor[dims 2, 1; f32]
y_pred.to_vec2::<f32>()?: [[-0.021972418], [1.3624947]]

[Training]
0 3.7258513
100 0.000080743244
200 0.000021977738
300 0.000014232694
400 0.00001018604
500 0.0000077515115
600 0.000006133396
700 0.000004985296
800 0.0000041298244
900 0.0000034713203
1000 0.0000029512994
model: Linear { weight: Tensor[dims 1, 1; f32], bias: Some(Tensor[1.0003569; f32]) }
model.weight(): Tensor[dims 1, 1; f32]
model.weight().to_vec2::<f32>()?: [[1.9979639]]
model.bias(): Some(Tensor[1.0003569; f32])

Ok(())

In [2]:
x = numpy.array([0, 1, 2, 3, 4]) 
y = x * 2 + 1

3. 이제 케라스로 인공신경망을 구현해보겠습니다.

In [3]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(1,input_shape=(1,)))
model.compile('SGD', 'mse')

2022-03-25 20:06:43.138044: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-25 20:06:43.139773: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-25 20:06:43.142855: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


4. 다음은 만든 모델을 주어진 데이터로 학습시킬 차례입니다. 

In [4]:
model.fit(x[:2], y[:2], epochs=1000, verbose=0)

2022-03-25 20:06:53.217764: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-25 20:06:53.223071: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3599995000 Hz


5. 이제 학습이 잘 되었는지 알아보겠습니다. 

In [5]:
y_pred = model.predict(x[2:]).flatten()
print('Targets:', y[2:])
print('Predictions:', y_pred)
print('Errors:', y[2:] - y_pred)

Targets: [5 7 9]
Predictions: [4.9872336 6.977996  8.968759 ]
Errors: [0.01276636 0.02200413 0.03124142]


---

### 1.3.3 전체 코드

In [6]:
# file: ex1_1_simple_exercise.py
# 1. 케라스 패키지 임포트
import keras 
import numpy

# 2. 데이터 지정
x = numpy.array([0, 1, 2, 3, 4]) 
y = x * 2 + 1

# 3. 인공신경망 모델링 
model = keras.models.Sequential()
model.add(keras.layers.Dense(1,input_shape=(1,)))
model.compile('SGD', 'mse')

# 4. 주어진 데이터로 모델 학습
model.fit(x[:2], y[:2], epochs=1000, verbose=0)

# 5.성능 평가
y_pred = model.predict(x[2:]).flatten()
print('Targets:', y[2:])
print('Predictions:', y_pred)
print('Errors:', y[2:] - y_pred)

Targets: [5 7 9]
Predictions: [4.943231 6.902153 8.861074]
Errors: [0.05676889 0.09784698 0.13892555]
